# General

This notebook is used to calculate and plot ensemble temperature figures for PMIP4 midHolocene paper.

### Import packages and define fucntions for calculations

In [1]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from netCDF4 import Dataset
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF


In [2]:


#Define pimp generations and model names
pmip_v4='PMIP4'
pmip_v3='PMIP3'
pmip={}
pmip['PMIP4']=['AWI-ESM-1-1-LR',
               'CESM2',
               'EC-Earth3-LR',
               'FGOALS-f3-L',
               'FGOALS-g3',
               'GISS-E2-1-G',
               'HadGEM3-GC31',
               'INM-CM4-8',
               'IPSL-CM6A-LR',
               'MIROC-ES2L',
               'MPI-ESM1-2-LR',
               'MRI-ESM2-0',
               'NESM3',
               'NorESM1-F',
               'NorESM2-LM',
               'UofT-CCSM-4']
pmip['PMIP3']=['BCC-CSM1-1',
               'CCSM4',
               'CNRM-CM5',
               'CSIRO-Mk3L-1-2',
               'CSIRO-Mk3-6-0',
               'EC-EARTH-2-2',
               'FGOALS-g2',
               'FGOALS-s2',
               'GISS-E2-R',
               'HadGEM2-CC',
               'HadGEM2-ES',
               'IPSL-CM5A-LR',
               'MIROC-ESM',
               'MPI-ESM-P',
               'MRI-CGCM3']
               
               
  




In [3]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict
    


# Create files

In [4]:
#Create a nc file to save cal. data
def createfile(filename):
    latS=np.arange(-89.5, 90., 1.0)
    lonS=np.arange(-179.5, 180., 1.0)
    d=Dataset(filename,'w',format='NETCDF4')
    d.createDimension('lon',360)  
    d.createDimension('lat',180)  
    d.createVariable("lon",'f',("lon"))  
    d.createVariable("lat",'f',("lat"))  

    d.variables['lat'][:]=latS     
    d.variables['lon'][:]=lonS 
    d.close()

## 180x360 Geo2D data files (individual models and ave and std)

In [11]:

def ensemble_mean(pmip_v,variable_name,experiment_name,filename):
    modellist=[]
    n=0
    average=0
    grid_1x1= xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90., 1.0)),
                         'lon': (['lon'], np.arange(-179.5, 180, 1.0))})
    gcm_dict=ensemble_members_dict(variable_name,experiment_name)
    for gcm in gcm_dict:
        if gcm in pmip[pmip_v]:
            modellist.append(gcm)
            this_file=xr.open_dataset(gcm_dict.get(gcm),decode_times=False)
            this_var=this_file[variable_name]
            this_regridder=xe.Regridder(this_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
            var_1x1=this_regridder(this_var)
            d=Dataset(filename,'a')
            d.createVariable(gcm,'f',("lat",'lon'))  
            d.variables[gcm][:]=var_1x1
            n=n+1
    d.close()      
    return modellist,average

In [14]:
filename='../outputs/netcdf/PMIP4_midHolocene_monsoon.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='monsoon_summer_rainrate'
createfile(filename)
a,b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_144x192_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc


In [16]:
filename='../outputs/netcdf/PMIP3_midHolocene_monsoon.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='monsoon_summer_rainrate'
createfile(filename)
a,b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [8]:
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    model_list=[]
#    data=np.zeros((16,90,180))
#    n=0
    d=Dataset(filename,'a')
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'piControl')
    grid_1x1= xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90., 1.0)),
                         'lon': (['lon'], np.arange(-179.5, 180, 1.0))})
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                model_list.append(gcm)
                d.createVariable(gcm,'f',("lat",'lon'))
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
                diff_1x1=this_regridder(diff) 
                d.variables[gcm][:]=diff_1x1                    
#                data[n]=diff_1x1
#                n=n+1
#    std=np.std(data,axis=0)
#    ave=np.average(data,axis=0)
#    d.createVariable('ave','f',("lat",'lon'))  
#    d.variables['ave'][:]=ave
#    d.createVariable('std','f',("lat",'lon'))  
#    d.variables['std'][:]=std
    d.close()    
    return model_list#,ave,std

In [9]:
def ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename):
    model_list=[]
#    data=np.zeros((15,90,180))
#    n=0
    d=Dataset(filename,'a')
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'piControl')
    grid_1x1= xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90., 1.0)),
                         'lon': (['lon'], np.arange(-179.5, 180, 1.0))})
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                model_list.append(gcm)
                d.createVariable(gcm,'f',("lat",'lon'))
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
                diff_1x1=this_regridder(diff) 
                d.variables[gcm][:]=diff_1x1                    
#                data[n]=diff_1x1
#                n=n+1
#    std=np.std(data,axis=0)
#    ave=np.average(data,axis=0)
#    d.createVariable('ave','f',("lat",'lon'))  
#    d.variables['ave'][:]=ave
#    d.createVariable('std','f',("lat",'lon'))  
#    d.variables['std'][:]=std
    d.close()    
    return model_list#,ave,std

In [10]:
filename='../outputs/netcdf/PMIP3_mh_diff_sst_ann.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='sst_spatialmean_ann'
createfile(filename)
liglist=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [11]:
filename='../outputs/netcdf/PMIP4_mh_diff_sst_ann.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='sst_spatialmean_ann'
createfile(filename)
liglist=ensemble_diffence_4(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_144x192_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [18]:
filename='../outputs/netcdf/PMIP3_mh_diff_mtco.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='tas_coldestmonth'
createfile(filename)
liglist=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [26]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_jja_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_jja'
createfile(filename)
liglist,pr_ave_jja,pr_std_jja=ensemble_diffence_4(pmip_v,experiment_name,variable_name,filename)
liglist


Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_180x288_90x180_peri.nc
Reuse existing file: bilinear_80x180_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_144x192_90x180_peri.nc
Reuse existing file: bilinear_120x180_90x180_peri.nc
Reuse existing file: bilinear_143x144_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [27]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_djf_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_djf'
createfile(filename)
liglist,pr_ave_djf,pr_std_djf=ensemble_diffence_4(pmip_v,experiment_name,variable_name,filename)
liglist



Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_180x288_90x180_peri.nc
Reuse existing file: bilinear_80x180_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_144x192_90x180_peri.nc
Reuse existing file: bilinear_120x180_90x180_peri.nc
Reuse existing file: bilinear_143x144_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [28]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_ann_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
createfile(filename)
liglist,tas_ave_ann,tas_std_ann=ensemble_diffence_4(pmip_v,experiment_name,variable_name,filename)

liglist


Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_180x288_90x180_peri.nc
Reuse existing file: bilinear_80x180_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_144x192_90x180_peri.nc
Reuse existing file: bilinear_120x180_90x180_peri.nc
Reuse existing file: bilinear_143x144_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [29]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_ann_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
createfile(filename)
liglist,pr_ave_ann,pr_std_ann=ensemble_diffence_4(pmip_v,experiment_name,variable_name,filename)
liglist



Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_180x288_90x180_peri.nc
Reuse existing file: bilinear_80x180_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_144x192_90x180_peri.nc
Reuse existing file: bilinear_120x180_90x180_peri.nc
Reuse existing file: bilinear_143x144_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_96x144_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [30]:

filename='../outputs/netcdf/PMIP4_mh_diff_2deg.nc'

createfile(filename)
d=Dataset(filename,'a')

d.createVariable('tas_ave_jja','f',("lat",'lon'))  
d.variables['tas_ave_jja'][:]=tas_ave_jja
d.createVariable('tas_std_jja','f',("lat",'lon'))  
d.variables['tas_std_jja'][:]=tas_std_jja

d.createVariable('tas_ave_djf','f',("lat",'lon'))  
d.variables['tas_ave_djf'][:]=tas_djf_ave
d.createVariable('tas_std_djf','f',("lat",'lon'))  
d.variables['tas_std_djf'][:]=tas_djf_std

d.createVariable('pr_ave_jja','f',("lat",'lon'))  
d.variables['pr_ave_jja'][:]=pr_ave_jja
d.createVariable('pr_std_jja','f',("lat",'lon'))  
d.variables['pr_std_jja'][:]=pr_std_jja

d.createVariable('pr_ave_djf','f',("lat",'lon'))  
d.variables['pr_ave_djf'][:]=pr_ave_djf
d.createVariable('pr_std_djf','f',("lat",'lon'))  
d.variables['pr_std_djf'][:]=pr_std_djf

d.createVariable('tas_ave_ann','f',("lat",'lon'))  
d.variables['tas_ave_ann'][:]=tas_ave_ann
d.createVariable('tas_std_ann','f',("lat",'lon'))  
d.variables['tas_std_ann'][:]=tas_std_ann

d.createVariable('pr_ave_ann','f',("lat",'lon'))  
d.variables['pr_ave_ann'][:]=pr_ave_ann
d.createVariable('pr_std_ann','f',("lat",'lon'))  
d.variables['pr_std_ann'][:]=pr_std_ann
d.close()

In [31]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_jja_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_jja'
createfile(filename)
liglist,tas_ave_jja,tas_std_jja=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Create weight file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Create weight file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Create weight file: bilinear_60x128_90x180_peri.nc
Create weight file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Create weight file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Create weight file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [32]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_djf_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_djf'
createfile(filename)
liglist,tas_djf_ave,tas_djf_std=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [33]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_jja_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_jja'
createfile(filename)
liglist,pr_ave_jja,pr_std_jja=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist

Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [34]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_djf_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_djf'
createfile(filename)
liglist,pr_ave_djf,pr_std_djf=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)
liglist



Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [35]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
createfile(filename)
liglist,tas_ave_ann,tas_std_ann=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)

liglist



Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [36]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
createfile(filename)
liglist,pr_ave_ann,pr_std_ann=ensemble_diffence_3(pmip_v,experiment_name,variable_name,filename)

liglist

Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_192x288_90x180_peri.nc
Reuse existing file: bilinear_128x256_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_56x64_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc
Reuse existing file: bilinear_60x128_90x180_peri.nc
Reuse existing file: bilinear_108x128_90x180_peri.nc
Reuse existing file: bilinear_90x144_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_145x192_90x180_peri.nc
Reuse existing file: bilinear_96x96_90x180_peri.nc
Reuse existing file: bilinear_64x128_90x180_peri.nc
Reuse existing file: bilinear_96x192_90x180_peri.nc
Reuse existing file: bilinear_160x320_90x180_peri.nc


['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MIROC-ESM',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [37]:

filename='../outputs/netcdf/PMIP3_mh_diff_2deg.nc'

createfile(filename)
d=Dataset(filename,'a')

d.createVariable('tas_ave_jja','f',("lat",'lon'))  
d.variables['tas_ave_jja'][:]=tas_ave_jja
d.createVariable('tas_std_jja','f',("lat",'lon'))  
d.variables['tas_std_jja'][:]=tas_std_jja

d.createVariable('tas_ave_djf','f',("lat",'lon'))  
d.variables['tas_ave_djf'][:]=tas_djf_ave
d.createVariable('tas_std_djf','f',("lat",'lon'))  
d.variables['tas_std_djf'][:]=tas_djf_std

d.createVariable('pr_ave_jja','f',("lat",'lon'))  
d.variables['pr_ave_jja'][:]=pr_ave_jja
d.createVariable('pr_std_jja','f',("lat",'lon'))  
d.variables['pr_std_jja'][:]=pr_std_jja

d.createVariable('pr_ave_djf','f',("lat",'lon'))  
d.variables['pr_ave_djf'][:]=pr_ave_djf
d.createVariable('pr_std_djf','f',("lat",'lon'))  
d.variables['pr_std_djf'][:]=pr_std_djf

d.createVariable('tas_ave_ann','f',("lat",'lon'))  
d.variables['tas_ave_ann'][:]=tas_ave_ann
d.createVariable('tas_std_ann','f',("lat",'lon'))  
d.variables['tas_std_ann'][:]=tas_std_ann

d.createVariable('pr_ave_ann','f',("lat",'lon'))  
d.variables['pr_ave_ann'][:]=pr_ave_ann
d.createVariable('pr_std_ann','f',("lat",'lon'))  
d.variables['pr_std_ann'][:]=pr_std_ann
d.close()

In [ ]:
# figure 2 , averaged bands

In [ ]:
modellist4= pd.read_csv('PMIP4_modellist.csv',skipinitialspace=True,header=0)['model']
modellist3= pd.read_csv('PMIP3_modellist.csv',skipinitialspace=True,header=0)['model']

In [ ]:
#Weighted array
#equator=0: cos
#equatro=90: sin
#Weighted array
#equator=0: cos
#equatro=90: sin
w=[]
a_LAT=180
a_LON=360
for l in range(0,a_LAT): #lat No.
        lat=l*180/a_LAT
        lats=np.deg2rad(lat)
        we=np.sin(lats)
        w.append(we)

a2=np.ones((a_LAT,a_LON)) #lat X lon
for r in range(0,a_LAT):
    for c in range(0,a_LON):
        a2[r,c]=a2[r,c]*w[r]



In [ ]:

def zonal(filename,pmip_v): 
    modellist=pd.read_csv('%s_modellist.csv'%pmip_v,skipinitialspace=True,header=0)['model']
    x=np.arange(0,181,30)
    this_file=xr.open_dataset(filename,decode_times=False)
    data4={}
    for m in modellist:
        data4[m]=[]
        for i in range(len(x)-1):
            men=this_file[m][x[i]:x[i+1]]
            xx=np.average(men,weights=a2[x[i]:x[i+1]])
            data4[m].append(xx)
    return data4



In [ ]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_ann.nc'
data=zonal(filename,'PMIP4')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP4_mh_diff_zonal_tas_ann.csv')

In [ ]:
filename='../outputs/netcdf/PMIP4_piControl_tas_ann.nc'
data=zonal(filename,'PMIP4')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP4_piControl_zonal_tas_ann.csv')

In [ ]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_ann.nc'
data=zonal(filename,'PMIP3')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP3_mh_diff_zonal_tas_ann.csv')

In [ ]:
filename='../outputs/netcdf/PMIP3_piControl_tas_ann.nc'
data=zonal(filename,'PMIP3')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP3_piControl_zonal_tas_ann.csv')

In [ ]:
# Fig 4 & 8
#tas, pr
#add signs

In [ ]:
def stipple(var):
    modellist= pd.read_csv('PMIP4_modellist.csv',skipinitialspace=True,header=0)['model']
    filename='../outputs/netcdf/PMIP4_mh_diff_%s.nc'%var
    stipple=np.zeros((180,360))
    d=Dataset(filename)
    averaged=d.variables['ave'][:]
    ave_sign=np.sign(averaged)
    for i,m in enumerate(modellist):
        modeldata=d.variables[m][:]
        model_sign=np.sign(modeldata)
        sign=abs(model_sign-ave_sign)
        stipple=stipple+sign
    d.close()
    agree_sign=stipple/2

    var_name='disagree_sign_%s'%var
    filename1='../outputs/netcdf/PMIP4_mh_diff.nc'
    dd=Dataset(filename1,'a')
    dd.createVariable(var_name,'f',("lat",'lon'))  
    dd.variables[var_name][:]=agree_sign
    dd.close()



In [ ]:
var1='pr_ann'
stipple(var1)

var2='tas_ann'
stipple(var2)

var3='pr_jja'
stipple(var3)

var4='tas_jja'
stipple(var4)

var5='pr_djf'
stipple(var5)

var6='tas_djf'
stipple(var6)



In [ ]:
# Monsoon files

In [13]:
def monsoon_cal(monsoon_name,experiment_name,pmip_v):
    monsoonlist=['NAF','EAS','AUSMC','NAMS','SAF','SAMS','SAS']
    data={}
    rainfall_name='monsoon_rain_%s' %monsoon_name
    area_name='monsoon_area_%s' %monsoon_name
    A_dict=ensemble_members_dict(rainfall_name,experiment_name)
    B_dict=ensemble_members_dict(rainfall_name,'piControl')
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_rain=expt_a_file[rainfall_name]
                expt_mean_rain=np.nanmean(expt_rain)
                expt_std_rain=np.nanstd(expt_rain)
                expt_area=expt_a_file[area_name]
                expt_mean_area=np.nanmean(expt_area)
                expt_std_area=np.nanstd(expt_area)
                expt_water=expt_rain*expt_area
                expt_mean_water=np.nanmean(expt_water)
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                pi_rain=expt_b_file[rainfall_name]
                pi_mean_rain=np.nanmean(pi_rain)
                pi_std_rain=np.nanstd(pi_rain)
                pi_area=expt_b_file[area_name]
                pi_mean_area=np.nanmean(pi_area)
                pi_std_area=np.nanstd(pi_area)
                pi_water=pi_rain*pi_area
                pi_mean_water=np.nanmean(pi_water)
                pav=(expt_mean_rain-pi_mean_rain)*100/pi_mean_rain
                psd=(expt_std_rain-pi_std_rain)*100/pi_std_rain
                aav=(expt_mean_area-pi_mean_area)*100/pi_mean_area
                asd=(expt_std_area-pi_std_area)*100/pi_std_area
                water=(expt_mean_water-pi_mean_water)*100/pi_mean_water
                data[gcm]=[pav,psd,aav,asd,water]
    f3='../outputs/csv/%s_%s.csv' %(pmip_v,monsoon_name)
    DD=pd.DataFrame(data)
    DD.to_csv(f3)
    return data

def monsoonchange(pmip_v,experiment_name):
    monsoonlist=['NAF','EAS','AUSMC','NAMS','SAF','SAMS','SAS',]
    modellistname='%s_modellist.csv'%pmip_v
    model_list4= pd.read_csv(modellistname,skipinitialspace=True,header=0)['model']
    pav_data4={}
    psd_data4={}
    aav_data4={}
    asd_data4={}
    water_data4={}
    for m in model_list4:
        pav_data4[m]=[]
        psd_data4[m]=[]
        aav_data4[m]=[]
        asd_data4[m]=[]
        water_data4[m]=[]
    for monsoon_name in monsoonlist:
        data4=monsoon_cal(monsoon_name,experiment_name,pmip_v)
        for m in model_list4:
            pav_data4[m].append(data4[m][0])
            psd_data4[m].append(data4[m][1])
            aav_data4[m].append(data4[m][2])
            asd_data4[m].append(data4[m][3])
            water_data4[m].append(data4[m][4])
    pd.DataFrame(pav_data4).to_csv('../outputs/csv/%s_pav.csv'%pmip_v)
    pd.DataFrame(aav_data4).to_csv('../outputs/csv/%s_aav.csv'%pmip_v)
    pd.DataFrame(psd_data4).to_csv('../outputs/csv/%s_psd.csv'%pmip_v)
    pd.DataFrame(asd_data4).to_csv('../outputs/csv/%s_asd.csv'%pmip_v)
    pd.DataFrame(water_data4).to_csv('../outputs/csv/%s_totwater.csv'%pmip_v)




In [14]:
monsoonchange('PMIP4','midHolocene')
monsoonchange('PMIP3','midHolocene')

In [24]:
def calaverage(pmip_v):
    varlist=['pav','psd','aav','asd','totwater']
    for v in varlist:    
        mon=v
        data=pd.read_csv('../outputs/csv/%s_%s.csv'%(pmip_v,mon),skipinitialspace=True,header=0)
        D=np.array(data)[:,1:]
        ave=np.average(D,axis=1)
        data['average']=ave
        data.to_csv('../outputs/csv/%s_%s.csv'%(pmip_v,mon))

In [25]:
calaverage('PMIP4')
calaverage('PMIP3')